In [77]:
#imports
import pandas as pd
import numpy as np
import pickle
from sklearn.cluster import DBSCAN, KMeans
import matplotlib.pyplot as plt

In [78]:
#import CSV file and create the database
#csv_filename = "age_specific_fertility_rates.csv"
csv_filename = "midyear_population_age_country_code.csv"
df = pd.read_csv(csv_filename)
#the line below is necessary only if the dataset is not ordered by year
#df = df.sort_values(['year'], ascending=[True])
snap_headers = np.unique(df['year']) #the filter that will define the snapshot range
#features = [i for i in df if 'fertility' in i]
features = [i for i in df if 'population' in i]
snapshots = [df[df['year'] == i] for i in snap_headers]
timeline = []

In [79]:
class Datapoint:
    def __init__(self, position, label=None):
        self.position = position
        self.label = label

    @property
    def position(self):
        return self.__position

    @position.setter
    def position(self, position):
        self.__position = position
    
    def get_fill(self):
        colors = [GREEN,BLUE,YELLOW, WHITE, CYAN, MAGENTA, GRAY]
        try:
            return colors[self.label]
        except:
            return RED
    
    def Print(self):
        print("Label: {}\nFeature Vector: {}\n".format(self.label, self.position))

    def draw(self, canvas):
        if len(self.position) != 2: return # does not render if data is not bidimensional
        #print(self.position)
        canvas.create_oval(int(self.position[0])-datapoint_radius, 
            self.position[1]-datapoint_radius, 
            self.position[0]+datapoint_radius, 
            self.position[1]+datapoint_radius, 
            fill=self.get_fill())

In [80]:
#util functions
def myplot(X, db, d):
    axes = plt.gca()
    axes.set_xlim([0,400])
    axes.set_ylim([0,400])
    
    plt.scatter(X[features[1]], X[features[2]], c=db, cmap='tab20', alpha=1, s =10)
    plt.savefig("./results/{}.png".format(str(np.unique(d['year'])[0])))
    plt.clf()

def save(filename, element):
    pickle.dump(element, open(filename, "wb"))

In [82]:
#run clustering for each snapshot
for d in snapshots:
    
    #print (datapoints[1][features])
    X = d[features].values
    #db = KMeans(n_clusters=int(np.sqrt(len(d))), random_state=0).fit_predict(X)
    db = DBSCAN(eps=5, min_samples=3, metric='euclidean', algorithm='auto', leaf_size=30, p=None, n_jobs=-1).fit_predict(X)
    timeline.append([Datapoint(X[i], db[i]) for i in range(len(X))])
        
    #print (X)
    #plt.scatter(X[features[0]], X[features[1]])
    #plt.savefig("{}.png".format(str(np.unique(d['year']))))
    #db = KMeans(n_clusters=int(np.sqrt(len(d))), random_state=0).fit_predict(X)
    

    #myplot(X, db, d)
    #print(np.unique(d['year']), np.unique(db), len(d))
save(csv_filename[:-4], timeline)

KeyboardInterrupt: 